In [6]:
# pair stance classification
import pandas as pd
import os, sys
import importlib
import numpy as np
sys.path.append("..")
import copy


import experimenter
from experimenter import evaluation
from experimenter.utils import modeling, training, data, text, utils
importlib.reload(experimenter)
importlib.reload(text)
importlib.reload(modeling)
importlib.reload(evaluation)
importlib.reload(training)
importlib.reload(data)
importlib.reload(utils)

configs = {"epochs": 25,
           "experiment_output_path": "local/final_training.json", 
           "log_interval": 1, "model_path": "local/model.state", 
           "processor": {
                "module": "experimenter.utils.data",
                "class": "PairStanceProvider",
                "params":{"input_path": "/Users/jkhouja/workspace/repo/arabic_media/data/for training/batch_0_to_15000_pairs_eq_weight_random.csv",
                          "seq_len": {'inp':[50,50], 'label':[1],'mask':[1]}, "batch_size": 4, "splits": [.7, .2, .1], "drop_last":True,  "shuffle": True, "vocab_path": "local/vocab.json"}},
           "model":{
                "module": "experimenter.utils.modeling",
                "class": "RNNPairModel",
                "params":{"embedding_dim": 20, "hidden_dim": 100, "num_classes": 3, "dropout": 0, "max_seq_len": {"eval": 1, "value": "config['processor']['params']['seq_len']['inp'][0]"}}},
           
            "evaluator":{
                "module": "experimenter.evaluation",
                "class": "ListEvaluator",
                "params": {"loss_f": [{"module": 'torch.nn', "class": 'CrossEntropyLoss', "params": {'reduction': 'none'}}],
                          "metrics_f": [{"module": 'experimenter.evaluation', "class": 'Accuracy', "params": {}}]}},
            "optimizer":{
                "module": "torch.optim",
                "class": "Adam",
                "params":{
                    "params": {
                    "eval": 1,
                    "value": "config['model']['model'].parameters()"},
                'lr':  0.001}}
           }



lm_configs = {"epochs": 5,
           "experiment_output_path": "local/final_training.json", 
           "log_interval": 1, "model_path": "local/model.state", 
           "processor": {
                "module": "experimenter.utils.data",
                "class": "LMProvider",
                "params":{"input_path": "/Users/jkhouja/workspace/repo/arabic_media/data/pairs/batch_0_to_15000_pairs_eq_weight_random.csv",
                          "seq_len": {'inp':[50], 'label':[50],'mask':[1]}, "batch_size": 4, "splits": [.1, .2, .1, .6], "drop_last":"False",  "shuffle": "True", "vocab_path": "local/vocab.json"}},
           "model":{
                "module": "experimenter.utils.modeling",
                "class": "RNNLMModel",
                "params":{"embedding_dim": 8, "hidden_dim": 30, "dropout": 0, "max_seq_len": {"eval": 1, "value": "config['processor']['params']['seq_len']['inp'][0]"}}},
           
            "evaluator":{
                "module": "experimenter.evaluation",
                "class": "ListEvaluator",
                "params": {"loss_f": [{"module": 'torch.nn', "class": 'CrossEntropyLoss', "params": {'reduction': 'none'}}]}},
            "optimizer":{
                "module": "torch.optim",
                "class": "Adam",
                "params":{
                    "params": {
                    "eval": 1,
                    "value": "config['model']['model'].parameters()"},
                'lr':  0.001}}
           }

multi_configs = {"epochs": 2,
           "experiment_output_path": "local/final_training.json", 
           "log_interval": 1, "model_path": "local/model.state", 
           "processor": {
                "module": "experimenter.utils.data",
                "class": "MultiLMPairProvider",
                "params":{"input_path": "/Users/jkhouja/workspace/repo/arabic_media/data/for training/batch_0_to_15000_pairs_eq_weight_random.csv",
                          "seq_len": {'inp':[50, 50], 'label':[50, 1],'mask':[1, 1]}, "batch_size": 4, "splits": [.7, .2, .1], "drop_last":True,  "shuffle": True, "vocab_path": "local/vocab.json"}},
           "model":{
                "module": "experimenter.utils.modeling",
                "class": "RNNMultiLMPairModel",
                "params":{"embedding_dim": 20, "hidden_dim": 100, "dropout": 0, "lm_classes": {"eval": 1, "value":"config['processor']['params']['vocab_size']"},  "num_classes":3 , "max_seq_len": {"eval": 1, "value": "config['processor']['params']['seq_len']['inp'][0]"}}},
           
            "evaluator":{
                "module": "experimenter.evaluation",
                "class": "ListEvaluator",
                "params": {"loss_f": [{"module": 'torch.nn', "class": 'CrossEntropyLoss', "params": {'reduction': 'none'}}, {"module": 'torch.nn', "class": 'CrossEntropyLoss', "params": {'reduction': 'none'}}],
                          "metrics_f": [{"module": 'experimenter.evaluation', "class": 'Dummy', "params": {}}, {"module": 'experimenter.evaluation', "class": 'Accuracy', "params": {}}]}},
            "optimizer":{
                "module": "torch.optim",
                "class": "Adam",
                "params":{
                    "params": {
                    "eval": 1,
                    "value": "config['model']['model'].parameters()"},
                'lr':  0.001}}
           }

#unlabeled_data = [{'inp':["يالت رجل", "قال رجلي"],'label':[['agree']],'mask':[1]}]*10

config =copy.deepcopy(multi_configs)

# Initialize two models
trainer_smart = training.BasicTrainer(config)

res_smart = trainer_smart.train_model()


All loaded data size:8754
Will create train, dev, test(s) splits
Total params: 58807
Starting training:


KeyboardInterrupt: 

In [41]:
%%writefile test.py

def test_ecoder_freezing():
    enc = experimenter.utils.text.encoder()
    sen1 = "This is a test sent"
    sen2 = "This is a test sent!?"  #Has 2 unseen characters
    _ = enc(sen1)
    len_before = len(enc.get_vocab())
    print(len_before)
    enc.freeze()
    _ = enc(sen2)
    assert len_before == len(enc.get_vocab())
    enc.unfreeze()
    _ = enc(sen2)
    print(len(enc.get_vocab()))
    assert len_before == len(enc.get_vocab()) - 2
    

test_ecoder_freezing()

11
Number of OOV: 2
13


In [261]:
# Evaluate on training
selected_data = trainer_smart.processor.get_data()

trainer_smart._evaluate_batches(selected_data[0])

[0.8669170476812541]

In [252]:
trainer_smart.evaluator.reset()
correct = 0
total = 0
res = []
for j, b in enumerate(selected_data[2]):
    #print(j)
    decoded_batch = trainer_smart.processor.decode(trainer_smart.processor._from_batch(b), list_input=True)
    #print(decoded_batch)
    res.append(trainer_smart.model(b))
    val_loss = trainer_smart.evaluator.update_batch(res[j])
    print(val_loss)
    print(trainer_smart.evaluator.get_metrics(res[j]))
    pred = trainer_smart.predict(decoded_batch, decode=False)
    current_corrects = sum([p['pred'][0] == p['label'][0] for p in pred])
    current_total = len(pred)
    print(current_corrects / current_total)
    #[print("||".join([str(p['pred'][0][0]), str(p['label'][0][0])])) for p in pred]
    print("===================")
    correct += current_corrects
    total += current_total
print(j, val_loss)
correct / total

[1.0]
[array(1., dtype=float32)]
1.0
[0.75]
[array(0.5, dtype=float32)]
0.5
[0.75]
[array(0.75, dtype=float32)]
0.75
[0.75]
[array(0.75, dtype=float32)]
0.75
[0.8]
[array(1., dtype=float32)]
1.0
[0.7916666666666666]
[array(0.75, dtype=float32)]
0.75
[0.7857142857142857]
[array(0.75, dtype=float32)]
0.75
[0.75]
[array(0.5, dtype=float32)]
0.5
[0.6944444444444444]
[array(0.25, dtype=float32)]
0.25
[0.725]
[array(1., dtype=float32)]
0.75
[0.7045454545454546]
[array(0.5, dtype=float32)]
0.75
[0.6875]
[array(0.5, dtype=float32)]
0.5
[0.6730769230769231]
[array(0.5, dtype=float32)]
0.5
[0.6785714285714286]
[array(0.75, dtype=float32)]
0.75
[0.7]
[array(1., dtype=float32)]
1.0
[0.6875]
[array(0.5, dtype=float32)]
0.5
[0.6764705882352942]
[array(0.5, dtype=float32)]
0.5
[0.6666666666666666]
[array(0.5, dtype=float32)]
0.75
[0.6578947368421053]
[array(0.5, dtype=float32)]
0.5
[0.675]
[array(1., dtype=float32)]
1.0
[0.6785714285714286]
[array(0.75, dtype=float32)]
0.25
[0.6704545454545454]
[arra

0.75
[0.6565040650406504]
[array(0.5, dtype=float32)]
0.5
[0.655241935483871]
[array(0.5, dtype=float32)]
0.0
[0.656]
[array(0.75, dtype=float32)]
0.75
[0.6587301587301587]
[array(1., dtype=float32)]
1.0
[0.6594488188976378]
[array(0.75, dtype=float32)]
0.75
[0.66015625]
[array(0.75, dtype=float32)]
0.75
[0.6608527131782945]
[array(0.75, dtype=float32)]
0.75
[0.6596153846153846]
[array(0.5, dtype=float32)]
0.5
[0.6603053435114504]
[array(0.75, dtype=float32)]
0.75
[0.6609848484848485]
[array(0.75, dtype=float32)]
0.75
[0.6616541353383458]
[array(0.75, dtype=float32)]
0.75
[0.664179104477612]
[array(1., dtype=float32)]
1.0
[0.6611111111111111]
[array(0.25, dtype=float32)]
0.5
[0.65625]
[array(0., dtype=float32)]
0.0
[0.656934306569343]
[array(0.75, dtype=float32)]
0.75
[0.6557971014492754]
[array(0.5, dtype=float32)]
0.75
[0.6528776978417267]
[array(0.25, dtype=float32)]
0.25
[0.6535714285714286]
[array(0.75, dtype=float32)]
0.75
[0.6524822695035462]
[array(0.5, dtype=float32)]
0.5
[0.6

0.6376146788990825

In [40]:
unlabeled_data = trainer_smart.processor.get_data()
#pred = trainer_smart._evaluate_batches(unlabeled_data[0])
pred = [p['pred'][1] == p['label'][1] for p in pred]
pred

[0.0, 0.3403890160183066]

In [121]:
correct = 0
total = 0
for b in selected_data[0]:
    decoded_batch = trainer_smart.processor.decode(trainer_smart.processor._from_batch(b), list_input=True)
    pred = trainer_smart.predict(decoded_batch, decode=False)
    correct += sum([p['pred'][0] == p['label'][0] for p in pred])
    total += len(pred)

#sum(pred) / len(pred) 
correct / total

0.42660550458715596

In [189]:
trainer_smart.evaluator.metrics_f[0](res[j]['pred'][0], res[j]['label'][0])
print(res[j]['pred'][0])
print(res[j]['label'][0])
[print("|".join([str(p['pred'][0][0]), str(p['label'][0][0])])) for p in pred]

tensor([[1],
        [2],
        [2],
        [1]])
tensor([[1],
        [0],
        [1],
        [1]])
2|1
1|1
2|0
0|1


[None, None, None, None]

In [196]:
res[j]['inp'][0]

tensor([[34, 13,  4, 15, 14,  6, 22, 11, 29,  2, 30,  3, 22,  6,  4,  2,  8, 15,
         22,  4,  3, 15,  9,  6,  2,  2, 24,  2,  4,  3,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0],
        [ 4, 34,  9, 28,  4, 28,  4,  9,  6,  4,  2,  7, 12,  4, 31, 30,  6, 21,
          8, 11, 41,  6, 34,  5, 12,  6, 15, 24, 11,  8,  6,  3, 34,  4, 27, 26,
          4,  9,  6,  4],
        [ 3, 11,  8, 15, 32,  6,  9, 36, 27, 16,  6,  4,  2,  9, 10, 22, 15, 27,
          4,  9,  6, 29,  4,  2, 28,  3,  2, 14,  6, 27, 15,  6,  9, 12, 13, 15,
          4, 35, 35,  6],
        [ 4,  2,  8, 11,  2,  4, 12,  6, 15, 31, 27, 32, 35, 35,  6, 11, 15,  7,
         32, 32,  6,  3, 13,  4, 24, 29, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0]])

In [197]:
pred[0]['inp'][0]

[3,
 11,
 8,
 15,
 32,
 6,
 9,
 36,
 27,
 16,
 6,
 4,
 2,
 9,
 10,
 22,
 15,
 27,
 4,
 9,
 6,
 29,
 4,
 2,
 28,
 3,
 2,
 14,
 6,
 27,
 15,
 6,
 9,
 12,
 13,
 15,
 4,
 35,
 35,
 0]

In [202]:
type(res[j]['inp'][0])

torch.Tensor

In [51]:
unlabeled_data = trainer_smart.processor.upload_data()
pred = trainer_smart.predict(unlabeled_data[:1000], decode=False)
pred = [p['pred'][0] == p['label'][0] for p in pred]
sum(pred) / len(pred) 

0.56

In [300]:
# Predict on unlabled data
def get_ent(preds):
    res = []
    for example in preds:
        ent = 0
        for p in example:
            ent += -p * np.log2(p)
        res.append(ent)
        
    return res

import random
top_n = 200
num_steps = range(10)
for step in num_steps:
    print("step {}".format(step))
    pred = trainer_smart.predict(unlabeled_data, decode=True)

    outs = [o for p in pred for o in p['meta']]
    ent = get_ent(outs)
    ent = np.array(ent)
    print(ent.mean())
    idx = (-ent).argsort()[:top_n]
    #print("selected examples based on entropy: ")
    #print(idx)
    filtered = [unlabeled_data[s] for s in idx]
    random_d = [unlabeled_data[s] for s in random.sample(range(len(unlabeled_data)), top_n)]

    # Update remaining
    print("Remaining data before / after filtering the learned batch")
    print(len(unlabeled_data))
    unlabeled_data = [s for i, s in enumerate(unlabeled_data) if i not in idx]
    print(len(unlabeled_data))

    #print(random)
    #print(filtered)

    #Retrain
    filtered_encoded = trainer_smart.processor(filtered, data_type='full', list_input= True, as_batches=True)
    random_encoded = trainer_smart.processor(random_d, data_type='full', list_input= True, as_batches=True)
    res_smart = trainer_smart.train_model([filtered_encoded, val_data, test_data])
    res_random = trainer_random.train_model([random_encoded, val_data, test_data])


step 0
1.4463678059057392
Remaining data before / after filtering the learned batch
5052
4852
Starting training:
Epoch: 11: Train loss (last batch): 1.0623605251312256, validation loss: 1.0581321716308594
Best model saved at: model.state
Epoch: 12: Train loss (last batch): 0.7871716618537903, validation loss: 1.0428305864334106
Best model saved at: model.state
Epoch: 13: Train loss (last batch): 1.1186940670013428, validation loss: 1.0667556524276733
Epoch: 14: Train loss (last batch): 1.3207297325134277, validation loss: 1.088484525680542
Epoch: 15: Train loss (last batch): 1.049099326133728, validation loss: 1.0591458082199097
Test loss: 1.0410854816436768
Starting training:
Epoch: 11: Train loss (last batch): 0.9175735712051392, validation loss: 1.0657908916473389
Best model saved at: model.state
Epoch: 12: Train loss (last batch): 1.0900952816009521, validation loss: 1.0391796827316284
Best model saved at: model.state
Epoch: 13: Train loss (last batch): 0.9136359691619873, validati

Best model saved at: model.state
Test loss: 0.9901295900344849
Starting training:
Epoch: 41: Train loss (last batch): 1.2643591165542603, validation loss: 1.0451629161834717
Best model saved at: model.state
Epoch: 42: Train loss (last batch): 1.170201063156128, validation loss: 1.052954912185669
Epoch: 43: Train loss (last batch): 0.956839919090271, validation loss: 1.0390136241912842
Best model saved at: model.state
Epoch: 44: Train loss (last batch): 1.3444180488586426, validation loss: 1.0519754886627197
Epoch: 45: Train loss (last batch): 0.9354345798492432, validation loss: 1.061521053314209
Test loss: 1.0727882385253906
step 7
1.3193084428125657
Remaining data before / after filtering the learned batch
3652
3452
Starting training:
Epoch: 46: Train loss (last batch): 0.9979891777038574, validation loss: 1.013409972190857
Best model saved at: model.state
Epoch: 47: Train loss (last batch): 1.0384202003479004, validation loss: 1.0270806550979614
Epoch: 48: Train loss (last batch): 0

In [330]:
ent.argmax()
ent[idx[100]]
ent[ent.argmax()]
unlabeled_data[idx[100]]['label'][0][0]
trainer_random.predict([unlabeled_data[idx[100]], unlabeled_data[idx[100]]])[0]['pred'][0][0]

'other'

In [320]:
res_smart['results']

{'during_training': {'1': {'train_loss': 1.0959333181381226,
   'val_loss': 1.0971243381500244},
  '2': {'train_loss': 1.1127394437789917, 'val_loss': 1.0927706956863403},
  '3': {'train_loss': 1.2162951231002808, 'val_loss': 1.0798826217651367},
  '4': {'train_loss': 1.1723943948745728, 'val_loss': 1.0638456344604492},
  '5': {'train_loss': 1.0028953552246094, 'val_loss': 1.0499619245529175},
  '6': {'train_loss': 1.090139627456665, 'val_loss': 1.0430418252944946},
  '7': {'train_loss': 0.9486572742462158, 'val_loss': 1.050020694732666},
  '8': {'train_loss': 1.1977633237838745, 'val_loss': 1.0444697141647339},
  '9': {'train_loss': 1.0246580839157104, 'val_loss': 1.0404095649719238},
  '10': {'train_loss': 0.6556094288825989, 'val_loss': 1.0469732284545898},
  '11': {'train_loss': 1.0623605251312256, 'val_loss': 1.0581321716308594},
  '12': {'train_loss': 0.7871716618537903, 'val_loss': 1.0428305864334106},
  '13': {'train_loss': 1.1186940670013428, 'val_loss': 1.0667556524276733},
 

In [171]:
res_random['results']

{'during_training': {'1': {'train_loss': 0.9932723045349121,
   'val_loss': 1.1460973024368286},
  '2': {'train_loss': 1.8655883073806763, 'val_loss': 0.9856749773025513},
  '3': {'train_loss': 0.5568922758102417, 'val_loss': 0.9643186330795288},
  '4': {'train_loss': 0.8071258664131165, 'val_loss': 0.9565287828445435},
  '5': {'train_loss': 0.5423922538757324, 'val_loss': 0.9556002020835876},
  '6': {'train_loss': 0.9349740743637085, 'val_loss': 0.9569883942604065},
  '7': {'train_loss': 1.3021361827850342, 'val_loss': 0.9552294015884399},
  '8': {'train_loss': 0.726514458656311, 'val_loss': 0.9544289708137512},
  '9': {'train_loss': 1.0989142656326294, 'val_loss': 0.955574095249176},
  '10': {'train_loss': 0.4896788001060486, 'val_loss': 0.955385684967041},
  '11': {'train_loss': 1.331813097000122, 'val_loss': 0.955652117729187},
  '12': {'train_loss': 1.322219729423523, 'val_loss': 0.956355631351471},
  '13': {'train_loss': 0.5277446508407593, 'val_loss': 0.9499547481536865},
  '14'

In [167]:
len(test_data)

252

In [86]:
t = trainer.evaluate(selected_data)
t

tensor(0.0958, grad_fn=<DivBackward0>)

In [90]:
!python ../run.py --config_file "../local/pairs.json"

Will create train, dev, test(s) splits
Total params: 2245
Starting training:
Epoch: 0: Train loss (last batch): 4.0922346115112305, validation loss: 4.063948154449463
Best model saved at: model.state
Epoch: 2: Train loss (last batch): 1.541860818862915, validation loss: 1.414445400238037
Best model saved at: model.state
tensor(1.3859, grad_fn=<DivBackward0>)


In [70]:
def test_chainer():
    tokenizer = text.tokenizer(sep=' ')
    enc = text.encoder(update_vocab=True)
    chain = text.chainer(funcs=[tokenizer, enc, enc.decode, tokenizer.detokenize])
    
    inp = "مرحبا هنا"
    assert inp == chain(inp, list_input=False)
    
    inp = ["hi_there man_", "how are you?"]
    assert inp == chain(inp, list_input=True)

test_chainer()

AttributeError: module 'experimenter.utils.text' has no attribute 'chainer'